In [3]:
print("a")

a


In [1]:
import pandas as pd
import numpy as np 
import matplotlib as plt
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('E:\@Career\AI\Technology\Main Use\ML\College _Content\Projects\Magic\magic04.data', header = None)
print(data)

             0         1       2       3       4         5         6   \
0       28.7967   16.0021  2.6449  0.3918  0.1982   27.7004   22.0110   
1       31.6036   11.7235  2.5185  0.5303  0.3773   26.2722   23.8238   
2      162.0520  136.0310  4.0612  0.0374  0.0187  116.7410  -64.8580   
3       23.8172    9.5728  2.3385  0.6147  0.3922   27.2107   -6.4633   
4       75.1362   30.9205  3.1611  0.3168  0.1832   -5.5277   28.5525   
...         ...       ...     ...     ...     ...       ...       ...   
19015   21.3846   10.9170  2.6161  0.5857  0.3934   15.2618   11.5245   
19016   28.9452    6.7020  2.2672  0.5351  0.2784   37.0816   13.1853   
19017   75.4455   47.5305  3.4483  0.1417  0.0549   -9.3561   41.0562   
19018  120.5135   76.9018  3.9939  0.0944  0.0683    5.8043  -93.5224   
19019  187.1814   53.0014  3.2093  0.2876  0.1539 -167.3125 -168.4558   

            7        8         9  10  
0      -8.2027  40.0920   81.8828  g  
1      -9.9574   6.3609  205.2610  g  
2     

<>:1: SyntaxWarning: invalid escape sequence '\@'
<>:1: SyntaxWarning: invalid escape sequence '\@'
C:\Users\KimoStore\AppData\Local\Temp\ipykernel_2440\73291539.py:1: SyntaxWarning: invalid escape sequence '\@'
  data = pd.read_csv('E:\@Career\AI\Technology\Main Use\ML\College _Content\Projects\Magic\magic04.data', header = None)


________________________________________________________________________________________________
Main_Way to make (Balance_class)

In [3]:
class_count=data[10].value_counts()

min_size=class_count.min()
print(min_size)

curr_labels=[]

for class_lab in class_count.index:
    curr_class=data[data[10] == class_lab]
    curr_class_ran =curr_class.sample(n=min_size , random_state=42)
    curr_labels.append(curr_class_ran)

data_no_bias=pd.concat(curr_labels)

data_shuffeled = data_no_bias.sample(frac=1 , random_state=42).reset_index(drop=True)

new_class_count=data_shuffeled[10].value_counts()
print(new_class_count)

data =data_shuffeled

print(data.isnull().sum()) # no null values

print(data.isnull())

6688
10
h    6688
g    6688
Name: count, dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64
          0      1      2      3      4      5      6      7      8      9   \
0      False  False  False  False  False  False  False  False  False  False   
1      False  False  False  False  False  False  False  False  False  False   
2      False  False  False  False  False  False  False  False  False  False   
3      False  False  False  False  False  False  False  False  False  False   
4      False  False  False  False  False  False  False  False  False  False   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
13371  False  False  False  False  False  False  False  False  False  False   
13372  False  False  False  False  False  False  False  False  False  False   
13373  False  False  False  False  False  False  False  False  False  False   
13374  False  False  False  False  False  False  False 

________________________________________________________________________________________________
Split Data 30/70

In [4]:
#spliting
train_set, test_set = train_test_split(data, test_size=0.3, random_state=42)

train_set.reset_index(drop=True, inplace=True)
test_set.reset_index(drop=True, inplace=True)

train_x=pd.DataFrame(train_set.iloc[:, 0 : 10])
train_y=pd.DataFrame(train_set.iloc[:, [10]])

test_x=pd.DataFrame(test_set.iloc[:, 0 : 10])
test_y=pd.DataFrame(test_set.iloc[:, [10]])

#rename to class
train_y.rename(columns={10: 'class'}, inplace=True)
test_y.rename(columns={10: 'class'}, inplace=True)

#print
print(train_set.shape)
print(test_set.shape)

print(train_y)
print(test_y)


(9363, 11)
(4013, 11)
     class
0        h
1        g
2        g
3        g
4        g
...    ...
9358     g
9359     g
9360     g
9361     h
9362     g

[9363 rows x 1 columns]
     class
0        h
1        g
2        g
3        h
4        g
...    ...
4008     g
4009     h
4010     h
4011     g
4012     g

[4013 rows x 1 columns]


________________________________________________________________________________________________
1st : Decision Tree

Code :

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
tree_clf=DecisionTreeClassifier(random_state=42)
tree_clf.fit(train_x,train_y)

Best parameters: {}


DecisionTreeClassifier(random_state=42)

Visualization :

In [29]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree_clf,
    out_file=r"E:\@Career\AI\Technology\Main Use\ML\College _Content\Projects\Magic\tree.dot",
    class_names=['g', 'h'], 
    rounded=True,
    filled=True
)

from graphviz import Source

Source.from_file(r"E:\@Career\AI\Technology\Main Use\ML\College _Content\Projects\Magic\tree.dot")


Performance :

In [33]:
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score, confusion_matrix

predict_class = tree_clf.predict(test_x)

accuracy = accuracy_score(test_y, predict_class)
print("Test Accuracy:", accuracy)

print("Predicted classes:")
print(predict_class)

cm = confusion_matrix(test_y, predict_class)
print("Confusion Matrix:")
print(cm)

precision = precision_score(test_y, predict_class, pos_label='g')
print("Precision:", precision)

recall = recall_score(test_y, predict_class, pos_label='g')
print("Recall:", recall)

f1 = f1_score(test_y, predict_class, pos_label='g')
print("F1 Score:", f1)


Test Accuracy: 0.7896835285322701
Predicted classes:
['h' 'h' 'g' ... 'h' 'h' 'h']
Confusion Matrix:
[[1597  379]
 [ 465 1572]]
Precision: 0.7744907856450048
Recall: 0.8081983805668016
F1 Score: 0.7909856364536899


________________________________________________________________________________________________
RandomForest

Training

In [31]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf=RandomForestClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],           
}

grid_search = GridSearchCV(rnd_clf, param_grid, cv=5, scoring='accuracy')

grid_search.fit(train_x, train_y)

print("Best parameters:", grid_search.best_params_)

best_tree_model = grid_search.best_estimator_

f:\Anaconda\Anaconda After SetUp\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda\Anaconda After SetUp\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda\Anaconda After SetUp\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
f:\Anaconda\Anaconda After SetUp\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the

Best parameters: {'n_estimators': 200}


Performance

In [32]:
predict_class = best_tree_model.predict(test_x)

accuracy = accuracy_score(test_y, predict_class)
print("Test Accuracy:", accuracy)

print("Predicted classes:")
print(predict_class)

cm = confusion_matrix(test_y, predict_class)
print("Confusion Matrix:")
print(cm)

precision = precision_score(test_y, predict_class, pos_label='g')
print("Precision:", precision)

recall = recall_score(test_y, predict_class, pos_label='g')
print("Recall:", recall)

f1 = f1_score(test_y, predict_class, pos_label='g')
print("F1 Score:", f1)

Test Accuracy: 0.8524794418141042
Predicted classes:
['h' 'g' 'g' ... 'h' 'g' 'g']
Confusion Matrix:
[[1741  235]
 [ 357 1680]]
Precision: 0.8298379408960915
Recall: 0.8810728744939271
F1 Score: 0.854688267059401


________________________________________________________________________________________________
Naïve Bayes

In [35]:
from sklearn.naive_bayes import GaussianNB

nb_clf = GaussianNB()
nb_clf.fit(train_x, train_y)


f:\Anaconda\Anaconda After SetUp\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GaussianNB()

performace

In [40]:
predict_class = nb_clf.predict(test_x)

accuracy = accuracy_score(test_y, predict_class)
print("Test Accuracy:", accuracy)

print("Predicted classes:")
print(predict_class)

cm = confusion_matrix(test_y, predict_class)
print("Confusion Matrix:")
print(cm)

precision = precision_score(test_y, predict_class, pos_label='g')
print("Precision:", precision)

recall = recall_score(test_y, predict_class, pos_label='g')
print("Recall:", recall)

f1 = f1_score(test_y, predict_class, pos_label='g')
print("F1 Score:", f1)

Test Accuracy: 0.6439073012708697
Predicted classes:
['h' 'g' 'g' ... 'g' 'g' 'g']
Confusion Matrix:
[[1766  210]
 [1219  818]]
Precision: 0.5916247906197655
Recall: 0.8937246963562753
F1 Score: 0.7119532352348317


________________________________________________________________________________________________
AdaBoost